In [ ]:
!pip install langchain langchain_openai langsmith langchain_community gradio

If you are using Jupyter notebook, follow the below instructions. Else skip this step and go to next step

**Open .env file in this folder and observe that we have configured OPENAI_API_KEY. Replace it with your own key or key given by me**

The Code in the below cell will load the .env file and set environment variables.

**Write the code in the below cell and execute it**

In [1]:
from dotenv import load_dotenv
import os
# Load environment variables from .env file
load_dotenv()


True

**Let us see if the model remembers previous messages**

If you execute the below code, you will understand that the model will not remember our previous messages by default

In [4]:
from langchain_openai import  ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

model = ChatOpenAI()
firstresponse = model.invoke([HumanMessage(content="Hi! I'm Siva")])

print(firstresponse.content)
secondresponse = model.invoke([HumanMessage(content="What's my name?")])

print(secondresponse.content)


Nice to meet you, Siva! How can I assist you today?
I'm sorry, I do not know your name as I am an AI assistant and do not have access to personal information.


In [3]:
secondresponse = model.invoke([HumanMessage(content="What's my name?")])

print(secondresponse.content)

I'm sorry, I don't know your name as I am an AI assistant and do not have access to your personal information.


**Now Let us Pass all the messages manually to the model and observe**

In [5]:
thirdresponse = model.invoke(
    [
        HumanMessage(content="Hi! I'm Siva"),
        AIMessage(content="Hello Siva! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

print(thirdresponse.content)

Your name is Siva.


**Now let us add chatmessagehistory manually**

In [6]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
chain = prompt | model
chain

ChatPromptTemplate(input_variables=['messages'], input_types={'messages': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core.mes

**Now, Let us create ChatMessageHistory , add messages and responses manually**

In [10]:
from langchain_community.chat_message_histories import ChatMessageHistory


chat_message_history = ChatMessageHistory()

chat_message_history.add_user_message(
    "Translate this sentence from English to French: I love programming."
)
response = chain.invoke(
    {"messages": chat_message_history.messages}
)
response


AIMessage(content='The translation of "I love programming" in French is "J\'adore programmer."', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 39, 'total_tokens': 58, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f56702e7-c848-4067-89a4-d957829c0b9f-0', usage_metadata={'input_tokens': 39, 'output_tokens': 19, 'total_tokens': 58, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

**Now, let us add the response from AI into chat message history**

**Then add our new usermessage also into chat message history**

**If you invoke the chain with all messages in chat message history, you will get response which makes us feel like LLM has memory**

**Understand and execute below code**

In [11]:
chat_message_history.add_ai_message(AIMessage(response.content))
#chat_message_history.messages

chat_message_history.add_user_message("What did i ask u?")

finalresponse =  chain.invoke({ "messages": chat_message_history.messages })

chat_message_history.add_ai_message(AIMessage(finalresponse.content))
(chat_message_history.messages)

[HumanMessage(content='Translate this sentence from English to French: I love programming.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The translation of "I love programming" in French is "J\'adore programmer."', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What did i ask u?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='You asked me to translate the sentence "I love programming" from English to French.', additional_kwargs={}, response_metadata={})]

**Now let us use RunnableWithMessageHistory. Let us see how it maintains message history automatically**

**Firstly, let us define a function which returns an instance of ChatMessageHistory based on session_id which is default configurable parameter**

In [13]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory

store = {}


def get_session_history(session_id: str) -> ChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

**Observe how we are passing get_session_history to RunnableWithMessageHistory Internally, RunnableWithMessageHistory will use get_session_history to get BaseChatMessageHistory for current session**



---





Understand the below code and execute it in a cell.
The below code goes through loop for 2 times asking you to enter your message.

For the first time, enter the message "My name is Siva"
Second time enter the message "What is my name?"

Once u get response, you will understand that how memory works


In [ ]:
from langchain_core.runnables.history import RunnableWithMessageHistory
count =0
config = {"configurable": {"session_id": "1122"}}

model_with_message_history = RunnableWithMessageHistory(model, get_session_history)

while(count < 2):
    content = input("Enter your message >> ")
    result = model_with_message_history.invoke(
        [HumanMessage(content=content)],
        config=config,
    )
    count = count + 1
    print(result.content)


model_with_message_history.get_session_history("1122").messages

Hello Nikhil, nice to meet you! How can I assist you today?
Your name is Nikhil.


[HumanMessage(content="i am nikhil'", additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hello Nikhil, nice to meet you! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 13, 'total_tokens': 30, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-476d666a-c177-487b-85ee-4fc371badaf0-0', usage_metadata={'input_tokens': 13, 'output_tokens': 17, 'total_tokens': 30, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 HumanMessage(content='what is my name?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Your name is Nikhil.', additional_kwargs

# Let us build a chat bot which has memory

**Understand the below code and execute it**

In [16]:
import gradio as gr
from langchain_core.runnables.history import RunnableWithMessageHistory

#session_id = input("Enter a session id >> ")

def predict(message, history):
   config = {"configurable": {"session_id": "1122"}}
   print(config)
   model_with_message_history = RunnableWithMessageHistory(model, get_session_history)

   response1= model_with_message_history.invoke(
        [HumanMessage(content=message)],
        config=config,
    ).content
   print(response1)

   print(model_with_message_history.get_session_history("1122").messages)
   return response1



gr.ChatInterface(predict).launch(debug=True)

c:\Users\Dell\Langchain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Dell\Langchain\venv\Lib\site-packages\gradio\chat_interface.py:334: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


{'configurable': {'session_id': '1122'}}
Hello Nikhil! How can I assist you today?
[HumanMessage(content="i am nikhil'", additional_kwargs={}, response_metadata={}), AIMessage(content='Hello Nikhil, nice to meet you! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 13, 'total_tokens': 30, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-476d666a-c177-487b-85ee-4fc371badaf0-0', usage_metadata={'input_tokens': 13, 'output_tokens': 17, 'total_tokens': 30, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), HumanMessage(content='what is my name?', additional_kwargs={}, re

**Now Let us use ChatPromptTemplate with RunnableWithMessageHistory**

Define a prompt using ChatPromptTemplate and then create a chain using it  as shown below



In [22]:
prompt = ChatPromptTemplate.from_messages(
    [  (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),  MessagesPlaceholder(variable_name="mychat_history"),
        ("human" ,"{myinput}")
    ]
)

chain =  prompt | model
chain

ChatPromptTemplate(input_variables=['language', 'mychat_history', 'myinput'], input_types={'mychat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], 

**Observe how we have Created  RunnableWithMessageHistory using the above chain. Also Observe input_messages_key and  history_messages_key**

In [23]:
model_with_message_history = RunnableWithMessageHistory(
    chain, get_session_history, input_messages_key="myinput", history_messages_key="mychat_history"
)
model_with_message_history


RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  mychat_history: RunnableBinding(bound=RunnableLambda(_enter_history), kwargs={}, config={'run_name': 'load_history'}, config_factories=[])
}), kwargs={}, config={'run_name': 'insert_history'}, config_factories=[])
| RunnableBinding(bound=RunnableLambda(_call_runnable_sync), kwargs={}, config={'run_name': 'check_sync_or_async'}, config_factories=[]), kwargs={}, config={'run_name': 'RunnableWithMessageHistory'}, config_factories=[]), kwargs={}, config={}, config_factories=[], get_session_history=<function get_session_history at 0x00000294A603B380>, input_messages_key='myinput', history_messages_key='mychat_history', history_factory_config=[ConfigurableFieldSpec(id='session_id', annotation=<class 'str'>, name='Session ID', description='Unique identifier for a session.', default='', is_shared=True, dependencies=None)])

**Understand how we are creating a chatbot using the below code and execute it in a cell**

In [25]:
def predict(message, history):
   config = {"configurable": {"session_id": "1298"}}
   print(config)


   response1= model_with_message_history.invoke(
        {"myinput": message,"language" : "Hindi"},
        config=config,
    ).content
   print(response1)

   print(model_with_message_history.get_session_history("1298").messages)
   return response1



gr.ChatInterface(predict).launch(debug=True)

c:\Users\Dell\Langchain\venv\Lib\site-packages\gradio\chat_interface.py:334: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


{'configurable': {'session_id': '1298'}}
नमस्ते, Nikhil! आपकी सेवा में कैसे मदद कर सकते हैं?
[HumanMessage(content='hi this is nikhil', additional_kwargs={}, response_metadata={}), AIMessage(content='你好，Nikhil！有什么可以帮到你的吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 35, 'total_tokens': 56, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c5964fd5-11c5-4d8d-bb32-57d6b78d662a-0', usage_metadata={'input_tokens': 35, 'output_tokens': 21, 'total_tokens': 56, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), HumanMessage(content='hello this is nikhil', additional_kwargs={}, response_metadata={})

**Let us see how to stream the result**

**Execute the below code in   cell and understand it**

In [ ]:
content = input(">> ")

streamtresult = model_with_message_history.stream(
       {"input": content,"language" : "English"},
        config=config,
    )
for token in streamtresult:
  print(token.content,end='|')

**Let is understand how to customize**

get_session_history functioned defined below uses a combination of user_id and conversation_id

Observe how we are customizing  RunnableWithMessageHistory using ConfigurableFieldSpec

**Execute the below cell and understand it**


In [27]:
from langchain_core.runnables import ConfigurableFieldSpec

store = {}


def get_session_history(user_id: str, conversation_id: str) -> BaseChatMessageHistory:
    if (user_id, conversation_id) not in store:
        store[(user_id, conversation_id)] = ChatMessageHistory()
    return store[(user_id, conversation_id)]


chain = prompt | model
model_with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="myinput", history_messages_key="mychat_history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="user_id", annotation=str, name="User ID",is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="conversation_id", annotation=str, name="Conversation ID",is_shared=True,
        ),
    ],
)

config = {"configurable": {"user_id": "123", "conversation_id": "1"}}
count =0
while(count < 2):
    content = input(">> ")
    result = model_with_message_history.invoke(
       {"myinput": content,"language" : "Hindi"},
        config=config,
    )
    count = count + 1

    print(result.content)
model_with_message_history.get_session_history("123","1").messages

नमस्ते निखिल, आपका स्वागत है। आपको कैसे मदद मिल सकती है?
आपका नाम निखिल है।


[HumanMessage(content='hi this is nikhil', additional_kwargs={}, response_metadata={}),
 AIMessage(content='नमस्ते निखिल, आपका स्वागत है। आपको कैसे मदद मिल सकती है?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 35, 'total_tokens': 91, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-cec89db8-a905-4f48-9f97-65b4eb0cf075-0', usage_metadata={'input_tokens': 35, 'output_tokens': 56, 'total_tokens': 91, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 HumanMessage(content='what is my name?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='आपका नाम निखिल है।', additional_kwargs

**Using FileChatMessageHistory to persist messages**

**Execute the below code to understand how chatmessage history is saved to a file**

In [36]:
from langchain_core.runnables import ConfigurableFieldSpec
from langchain_community.chat_message_histories import FileChatMessageHistory

store = {}


def get_session_history(user_id: str, conversation_id: str) -> BaseChatMessageHistory:
    if (user_id, conversation_id) not in store:
        store[(user_id, conversation_id)] = FileChatMessageHistory(
            user_id + conversation_id + "savedconversationmessages.json"
        )
    return store[(user_id, conversation_id)]


chain = prompt | model
model_with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="myinput",
    history_messages_key="mychat_history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name="User ID",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="conversation_id",
            annotation=str,
            name="Conversation ID",
            is_shared=True,
        ),
    ],
)

config = {"configurable": {"user_id": "123", "conversation_id": "1"}}
count = 0
while count < 2:
    content = input(">> ")
    result = model_with_message_history.invoke(
        {"myinput": content, "language": "Hindi"},
        config=config,
    )
    count = count + 1

    print(result.content)
model_with_message_history.get_session_history("123", "1").messages

नमस्ते जॉन, आपकी सेवा में। आपको कैसे मदद मिल सकती है?
आपका नाम जॉन है।


[HumanMessage(content='hi this is john', additional_kwargs={}, response_metadata={}),
 AIMessage(content='नमस्ते जॉन, आपकी सेवा में। आपको कैसे मदद मिल सकती है?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 33, 'total_tokens': 86, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-da01d3cc-7f89-48a3-85bc-1bcc806b3738-0', usage_metadata={'input_tokens': 33, 'output_tokens': 53, 'total_tokens': 86, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 HumanMessage(content='what is my name?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='आपका नाम जॉन है।', additional_kwargs={'refu

In [37]:
store

{('123',
  '1'): <langchain_community.chat_message_histories.file.FileChatMessageHistory at 0x294b8bdc790>}